In [4]:
import threading
import socket
import pickle

In [55]:
class ManagerPeer:
    def __init__(self, ip):
        self.peer_set = set()
        self.MANAGER_IP = ip
        self.MANAGER_PORT = 8800
        self.server_port = 5500
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server_thread = threading.Thread(target=self.start_server)
        self.server_thread.daemon = True
        self.server_thread.start()
        self.stop_server = False
        
    def start_server(self):
        self.server_socket.bind((self.MANAGER_IP, self.MANAGER_PORT))
        self.server_socket.listen(10)
        while True:
            if self.stop_server:
                break

            client_peer, address = self.server_socket.accept()
            print(f"incoming connection from {address}")
            recieved_request = client_peer.recv(4096).decode('utf-8')            
            ip = recieved_request.split()[1]

            if recieved_request.startswith("add"):
                self.inform_peers(recieved_request)
                client_peer.send(pickle.dumps(self.peer_set))
                self.peer_set.add(ip)
                print(f"added {ip}")
                
            elif recieved_request.startswith('remove'):
                self.peer_set.remove(ip)
                self.inform_peers(recieved_request)
                print(f"removed {ip}")
                
            client_peer.close()
        self.server_socket.close()
    
    def _stop_server(self):
        self.stop_server = True
    
    def inform_peers(self, message):
        operation = message.split()[0]
        for peer in self.peer_set:
            client_socket = socket.socket()
            client_socket.connect((peer, self.server_port))
            client_socket.send(f"{operation}-peer {peer}".encode('utf-8'))
            print(f"Sent to {peer}")
            client_socket.close()
    
    def __repr__(self):
        return f'{self.MANAGER_IP} {self.MANAGER_PORT}\nPeers: '+' '.join(self.peer_set)

In [56]:
manager_peer = ManagerPeer('192.168.1.109')

In [48]:
print(manager_peer.peer_set) 

set()
('192.168.43.228', 49905)
added 192.168.43.228
('192.168.43.228', 49906)
removed 192.168.43.228


In [49]:
manager_peer._stop_server()

In [50]:
manager_peer

In [57]:
s = set(['a','b'])

In [59]:
'hey'+' '.join(s)

'heyb a'